In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import time

**Collecting Mars news**

In [ ]:
!which chromedriver

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://mars.nasa.gov/news/'

In [4]:
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
news_title = soup.find('ul', class_='item_list ').find('li', class_='slide').find('div', class_='content_title')\
.find('a').get_text()

In [7]:
news_p = soup.find('ul', class_='item_list ').find('li', class_='slide').\
find('div', class_='article_teaser_body').get_text()

In [8]:
print(news_title)
print(news_p)

NASA's Curiosity Rover Aims to Get Its Rhythm Back
Rover engineers at JPL will try to restore percussive drilling on Mars this week, part of a larger series of tests that will last through summer.


**Getting Mars image link**

In [9]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [10]:
browser.visit(url)

***Open full image***

In [11]:
browser.find_by_css('div[class="default floating_text_area ms-layer"]').find_by_css('footer')\
.find_by_css('a[class="button fancybox"]').click()

***Open More Info***

In [12]:
browser.find_by_css('div[id="fancybox-lock"]').find_by_css('div[class="buttons"]')\
.find_by_css('a[class="button"]').click()

In [13]:
featured_image_url = browser.find_by_css('div[id="page"]').find_by_css('section[class="content_page module"]')\
.find_by_css('figure[class="lede"]').find_by_css('a')['href']

In [14]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19324_hires.jpg


**Scraping Mars weather**

In [3]:
url = 'https://twitter.com/marswxreport?lang=en'

In [4]:
browser.visit(url)

In [ ]:
# all_tweets = browser.find_by_css('ol[id="stream-items-id"]')

In [5]:
tweet_features = 'Sol' and 'high' and 'low' and 'pressure' and 'hPa' and 'daylight'

In [16]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [33]:
# soup.find_all('li', class_="js-stream-item stream-item stream-item ")

In [40]:
# for tweet in soup.find_all('li', class_="js-stream-item stream-item stream-item "):
#     print(tweet.find('div', class_='js-tweet-text-container').find('p').text)

In [38]:
for tweet in soup.find_all('li', class_="js-stream-item stream-item stream-item "):
    if tweet_features in tweet.find('div', class_='js-tweet-text-container').find('p').text:
        mars_weather = tweet.find('div', class_='js-tweet-text-container').find('p').text
        break

In [39]:
print(mars_weather)

Sol 2056 (May 19, 2018), Sunny, high 5C/41F, low -74C/-101F, pressure at 7.40 hPa, daylight 05:20-17:20


**Gathering Mars facts**

In [2]:
url = 'http://space-facts.com/mars/'

In [3]:
tables = pd.read_html(url)

In [4]:
df = tables[0]
df.columns = ['Parameter', 'Value']
df = df.set_index('Parameter')
df

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [5]:
html_table = df.to_html()
html_table.strip()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Parameter</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

**Hemispheres data**

In [24]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [26]:
browser.visit(url)

In [27]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [28]:
spheres = soup.find('div', class_='collapsible results').find_all('div', class_='item')

In [29]:
hemisphere_image_urls = []
for x in range(len(spheres)):
    title = spheres[x].find('div', class_="description").find('h3').text
    browser.find_by_css('div[class="collapsible results"]').find_by_css('div[class="item"]')[x]\
    .find_by_css('div[class="description"]').find_by_css('a').click()
    for img in browser.find_by_css('div[class="downloads"]').find_by_css('a'):
        if ('Original' in img.text):
            img_url = img['href']
#         if ('Sample') in img.text:
#             img_url = img['href']
    browser.click_link_by_partial_text('Back')
    dic = {'title': title, 'img_ulr': img_url}
    hemisphere_image_urls.append(dic)
    time.sleep(3)
    print(title)
    print(img_url)

Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif
Schiaparelli Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif
Syrtis Major Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif
Valles Marineris Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif


In [30]:
hemisphere_image_urls

[{'img_ulr': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_ulr': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_ulr': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_ulr': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif',
  'title': 'Valles Marineris Hemisphere Enhanced'}]